#### Setup

In [1]:
%pip install scikit-learn pandas numpy matplotlib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.chdir('../')

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


#### Get data and Split into train test

In [ ]:
#import and combine cluster data with preprocessed data
cluster_data = pd.read_csv(r'Data/Seattle_Building_Clusters.csv')
preprocessed_data = pd.read_csv(r'Data/Building_Data_Preprocessed.csv')

preprocessed_data = preprocessed_data.assign(GHGIntensityCluster = cluster_data['GHGIntensityCluster']) #move clusters to preprocessed data

preprocessed_data = preprocessed_data.drop('BuildingName', axis = 1) #drop string columns
preprocessed_data = preprocessed_data.drop('Address', axis = 1)
preprocessed_data = preprocessed_data.drop('Latitude', axis = 1) #Drop columns with negative values
preprocessed_data = preprocessed_data.drop('Longitude', axis = 1)#Drop columns with negative values


preprocessed_data = preprocessed_data.dropna()

#very ugly, i might add this to the preprocessing notebook so that the clusters are included in the data

X = preprocessed_data.drop('GHGIntensityCluster', axis=1)
y = preprocessed_data['GHGIntensityCluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


#### Multi-nomial Naive Bayes

In [24]:
mnnb = MultinomialNB()

# Parameters for tuning
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0, 20, 50, 100],
    'force_alpha': [True, False],
}

# Grid search the parameters to hopefully improve accuracy
grid_search = GridSearchCV(mnnb, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Make predictions
y_pred = grid_search.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred) #accuracy is the % of correct predictions made
print(f'Multinomial Naive Bayes Accuracy: {accuracy:.4f}')

print('Multinomial Classification Report:')
print(classification_report(y_test, y_pred))

print('Multinomial Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Multinomial Naive Bayes Accuracy: 0.4933
Multinomial Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.75      0.66       399
           1       0.27      0.40      0.32        81
           2       0.29      0.02      0.03       121
           3       0.30      0.22      0.26       143

    accuracy                           0.49       744
   macro avg       0.36      0.35      0.32       744
weighted avg       0.45      0.49      0.44       744

Multinomial Confusion Matrix:
[[301  42   4  52]
 [ 46  32   1   2]
 [ 71  29   2  19]
 [ 94  17   0  32]]


#### Categorical Naive Bayes Classification

In [26]:
canb = CategoricalNB()

# Parameters for tuning
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0, 20, 50, 100],
    'force_alpha': [True, False],
}

# Grid search the parameters to hopefully improve accuracy
grid_search = GridSearchCV(canb, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Predict
y_pred = grid_search.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f'Compliment Naive Bayes Accuracy: {accuracy:.2f}')

print('Compliment Classification Report:')
print(classification_report(y_test, y_pred))

print('Compliment Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))



/Volumes/T7/DATA422_Fall2024_Team5/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Volumes/T7/DATA422_Fall2024_Team5/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Volumes/T7/DATA422_Fall2024_Team5/.venv/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 455, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Volumes/T7/DATA422_Fall2024_Team5/.venv/lib/python3.12/site-packages/sklearn/base.py", line 764, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "/Volume

IndexError: index 50886 is out of bounds for axis 1 with size 50875

#### Write Up
These models both have pretty bad accuracy. Naive Bayes classification models assume that all features are independent and it is likely that these 